# Lab : Image Classification using Convolutional Neural Networks

At the end of this laboratory, you would get familiarized with

*   Creating deep networks using Keras
*   Steps necessary in training a neural network
*   Prediction and performance analysis using neural networks

---

# **In case you use a colaboratory environment**
By default, Colab notebooks run on CPU.
You can switch your notebook to run with GPU.

In order to obtain access to the GPU, you need to choose the tab Runtime and then select “Change runtime type” as shown in the following figure:

![Changing runtime](https://miro.medium.com/max/747/1*euE7nGZ0uJQcgvkpgvkoQg.png)

When a pop-up window appears select GPU. Ensure “Hardware accelerator” is set to GPU.

# **Working with a new dataset: CIFAR-10**

The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. More information about CIFAR-10 can be found [here](https://www.cs.toronto.edu/~kriz/cifar.html).

In Keras, the CIFAR-10 dataset is also preloaded in the form of four Numpy arrays. x_train and y_train contain the training set, while x_test and y_test contain the test data. The images are encoded as Numpy arrays and their corresponding labels ranging from 0 to 9.

Your task is to:

*   Visualize the images in CIFAR-10 dataset. Create a 10 x 10 plot showing 10 random samples from each class.
*   Convert the labels to one-hot encoded form.
*   Normalize the images.




In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix
import cv2
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Convert labels to one-hot encoding

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)



# Normalize the images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

In [ ]:
# Your code here :
# Load CIFAR-10 dataset
# Plot 10 random images per class
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
fig, axes = plt.subplots(10, 10, figsize=(10, 10))
plt.subplots_adjust(hspace=0.5)

for class_idx in range(10):
    indices = np.where(y_train.argmax(axis=1) == class_idx)[0]
    sample_indices = np.random.choice(indices, 10, replace=False)

    for j, img_idx in enumerate(sample_indices):
        ax = axes[class_idx, j]
        ax.imshow(x_train[img_idx])
        ax.axis('off')
        if j == 0:
            ax.set_ylabel(class_names[class_idx], fontsize=10, rotation=90, labelpad=10)

plt.show()

## Define the following model (same as the one in tutorial)

For the convolutional front-end, start with a single convolutional layer with a small filter size (3,3) and a modest number of filters (32) followed by a max pooling layer.

Use the input as (32,32,3).

The filter maps can then be flattened to provide features to the classifier.

Use a dense layer with 100 units before the classification layer (which is also a dense layer with softmax activation).

In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
# Your code here :
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(32,32,3)),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(100,activation='relu'),
    Dense(10,activation='softmax')
])
model.summary()

*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 50 epochs with a batch size of 512.

In [ ]:
# Your code here :
batch_size = 512
epochs = 50

model.compile(loss="categorical_crossentropy", optimizer="SGD", metrics=["accuracy"])

model.fit(x_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, validation_split=0.1)


*   Plot the cross entropy loss curve and the accuracy curve

In [ ]:
# Your code here :
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Defining Deeper Architectures: VGG Models

*   Define a deeper model architecture for CIFAR-10 dataset and train the new model for 50 epochs with a batch size of 512. We will use VGG model as the architecture.

Stack two convolutional layers with 32 filters, each of 3 x 3.

Use a max pooling layer and next flatten the output of the previous layer and add a dense layer with 128 units before the classification layer.

For all the layers, use ReLU activation function.

Use same padding for the layers to ensure that the height and width of each layer output matches the input


In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
# Your code here :
model_VGG = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),  # First Conv Layer
    Conv2D(32, (3,3), activation='relu', padding='same'),  # Second Conv Layer
    MaxPooling2D(pool_size=(2,2)),  # Max Pooling Layer
    Flatten(),  # Flatten Layer
    Dense(128, activation='relu'),  # Fully Connected Layer with 128 neurons
    Dense(10, activation='softmax')  # Output Layer with 10 classes
])

*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 50 epochs with a batch size of 512.

In [ ]:
# Your code here :
model_VGG.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])
history = model_VGG.fit(x_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, validation_split=0.1)


*   Compare the performance of both the models by plotting the loss and accuracy curves of both the training steps. Does the deeper model perform better? Comment on the observation.


In [ ]:
# Your code here :
plt.plot(model_VGG.history.history['loss'])
plt.plot(model_VGG.history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.plot(model_VGG.history.history['accuracy'])
plt.plot(model_VGG.history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

**Comment on the observation**

VGG reduces both training and validation loss at a similar rate, meaning it generalizes better than the simple CNN.

...

*   Use predict function to predict the output for the test split
*   Plot the confusion matrix for the new model and comment on the class confusions.


In [ ]:
# Your code here :
y_pred_probabilities = model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probabilities, axis=1)
y_true_classes = np.argmax(y_test_one_hot, axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()



**Comment here :**

it seems the model confuse with class bird and cat ,deer ,dog but work better with frog and horse , ship and truck class
...

*    Print the test accuracy for the trained model.

In [ ]:
# Your code here :
test_loss, test_accuracy = model.evaluate(x_test, y_test_one_hot)
print(f"Test accuracy: {test_accuracy:.4f}")

## Define the complete VGG architecture.

Stack two convolutional layers with 64 filters, each of 3 x 3 followed by max pooling layer.

Stack two more convolutional layers with 128 filters, each of 3 x 3, followed by max pooling, followed by two more convolutional layers with 256 filters, each of 3 x 3, followed by max pooling.

Flatten the output of the previous layer and add a dense layer with 128 units before the classification layer.

For all the layers, use ReLU activation function.

Use same padding for the layers to ensure that the height and width of each layer output matches the input

*   Change the size of input to 64 x 64.

In [ ]:
from keras.backend import clear_session
clear_session()

In [ ]:
# Your code here :
# Build the deeper VGG-like model
vgg_deep_model = Sequential()
# Add layers to the model (example of a deeper VGG-like structure)
vgg_deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)))
vgg_deep_model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
vgg_deep_model.add(MaxPooling2D(pool_size=(2, 2)))

vgg_deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vgg_deep_model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
vgg_deep_model.add(MaxPooling2D(pool_size=(2, 2)))

vgg_deep_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
vgg_deep_model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
vgg_deep_model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output of the last convolutional layer
vgg_deep_model.add(Flatten())  # This will flatten the 4x4x256 output to a vector of length 4096
vgg_deep_model.add(Dense(512, activation='relu'))
vgg_deep_model.add(Dense(10, activation='softmax'))  # 10 classes for CIFAR-10

*   Compile the model using categorical_crossentropy loss, SGD optimizer and use 'accuracy' as the metric.
*   Use the above defined model to train CIFAR-10 and train the model for 10 epochs with a batch size of 512.
*   Predict the output for the test split and plot the confusion matrix for the new model and comment on the class confusions.

In [ ]:
# Your code here :
# Compile the model
vgg_deep_model.compile(loss='categorical_crossentropy',
                       optimizer=SGD(),
                       metrics=['accuracy'])

# Train the model
vgg_deep_history = vgg_deep_model.fit(x_train, y_train,
                                      epochs=10,
                                      batch_size=512,
                                      validation_data=(x_test, y_test))

In [ ]:
y_pred_probabilities = vgg_deep_model.predict(x_test)
y_pred_classes = np.argmax(y_pred_probabilities, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true_classes, y_pred_classes)

plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()

# Understanding deep networks

*   What is the use of activation functions in network? Why is it needed?
*   We have used softmax activation function in the exercise. There are other activation functions available too. What is the difference between sigmoid activation and softmax activation?
*   What is the difference between categorical crossentropy and binary crossentropy loss?

**Write the answers below :**

1 - Use of activation functions:
 to normalize the input



_

2 - Key Differences between sigmoid and softmax:
Sigmoid: Outputs a probability between 0 and 1 for each neuron
Softmax: Converts outputs into a probability distribution summing to 1. Used for multi-class classification.


_

3 - Key Differences between categorical crossentropy and binary crossentropy loss:


_
binary crossentropy using for binary classfication categorical crossentropy use for multi class classfication